[View in Colaboratory](https://colab.research.google.com/github/SeaGoUnlimited/SB-Water-Quality/blob/master/NiklasCode10_19_18.ipynb)

In [3]:
! pip install geopandas

    100% |████████████████████████████████| 901kB 8.7MB/s 
    100% |████████████████████████████████| 15.8MB 2.0MB/s 
    100% |████████████████████████████████| 4.4MB 7.9MB/s 
    100% |████████████████████████████████| 1.5MB 14.1MB/s 
    100% |████████████████████████████████| 81kB 24.3MB/s 
  Running setup.py bdist_wheel for pyproj ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/89/77/ec/a537585d1022dafde0317dd19d33c4a30d4ee61e19f25ebd8e
  Running setup.py bdist_wheel for munch ... - done
  Stored in directory: /root/.cache/pip/wheels/db/bf/bc/06a3e1bfe0ab27d2e720ceb3cff3159398d9264

Importing the necessary modules. 

In [0]:
import os, json
import pandas 
import numpy as np
import geopandas
from shapely.geometry import Point, LineString

Loading json file, extracting the track data, and converting it into a geodataframe.

In [0]:
#Create a function that loads json by filename
#We want two tables, one with track and one with sightings
def load_json(filename):
  with open(filename) as f:
    data = json.load(f)  
  data['track'] = extract_track(data)  
  trip = pandas.DataFrame([data])
  sightings = extract_sightings(data)
  return sightings,  trip

In [0]:
def extract_track(data):
  coordinates = []
  for trkpt in data['track']['gpx']['trk']['trkseg']['trkpt']:
    coordinate = (float(trkpt['@lon']), float(trkpt['@lat']))
    coordinates.append(coordinate)
  line = LineString(coordinates).wkt
  return line

Extracting the sightings data  and putting it into a pandas dataframe. Then created a new datafram which is merged by date. This dataframe is transformed to a point which is stored in the new column, 'coordinates'. Then the dataframe is converted to a geodataframe.

In [0]:
#Create a function to extract sightings data
def extract_sightings(data):
  sightings = data['sightings']
  df = pandas.DataFrame()
  for sighting in sightings:
    df = df.append(sighting, ignore_index=True)    
    #create a new data frame merged by date, pulling all the first occurances in the new table
  df = df.groupby(by='create_date').first()
  df['coordinates'] =  list(zip(df.lon, df.lat))
  df['coordinates'] = df['coordinates'].apply(Point)
  gdf = geopandas.GeoDataFrame(df, geometry='coordinates')
  return gdf

Main function needed to create a loop.

In [0]:
def load_folder(folder_name):
  sighthings = pandas.DataFrame()
  trips = pandas.DataFrame()
  for filename in os.listdir(folder_name)  :
    sighting, trip = loadjson(filename=filename)
    sightings.append(sighting)
    trips.append(trip)    

We have extracted the trip data and the sightings...

In [33]:
sightings, trip = load_json(filename="trip_58.json")
rip

,CINMS Vessel,CINMS Weather,Comments,Observer Names,create_date,end_date,sightings,start_date,track
0,R/V Fulmar,"[{'Beaufort Scale': 2, 'create_date': '2013-07...",,Kate Trimlett and ACCESS science team,2013-07-24 14:26:09+00:00,2013-07-24 22:18:37+00:00,"[{'create_date': '2013-07-24 19:07:52+00:00', ...",2013-07-24 14:32:36+00:00,"LINESTRING (-122.504008 37.656192, -122.504068..."


In [48]:
#os.cwd()
os.listdir('/content/')
#load_folder('/content/*.json')


['.config', 'sample_data', 'trip_10005.json', 'trip_58.json']